In [1]:
import pandas as pd

In [2]:
import re
import jieba


def processing(text):
    """
    数据预处理, 可以根据自己的需求进行重载
    """
    # 数据清洗部分
    text = re.sub("\{%.+?%\}", " ", text)  # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)  # 去除 @xxx (用户名)
    text = re.sub("【.+?】", " ", text)  # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    text = re.sub("\u200b", " ", text)  # '\u200b'是这个数据集中的一个bad case, 不用特别在意
    # 分词
    words = [w for w in jieba.lcut(text) if w.isalpha()]
    # 对否定词`不`做特殊处理: 与其后面的词进行拼接
    while "不" in words:
        index = words.index("不")
        if index == len(words) - 1:
            break
        words[index: index + 2] = ["".join(words[index: index + 2])]  # 列表切片赋值的酷炫写法
    # 用空格拼接成字符串
    result = " ".join(words)
    return result


def load_corpus(aaa):
    """
    加载语料库
    """
    data = []
    for line in aaa:
        content = processing(line)
        data.append(content)
    return data


In [3]:
df = pd.read_csv('moredata_label.csv')
df.head()

content  score
0  槟榔谷的大门，一左一右两个大牛头，霸气侧漏的感觉溢于言表。拿着预订短信和身份证到售票处办理了...      1
1  海南保亭的黎苗族风情景区，可以看到他们的艺术，居住环境，房屋，传统表演！大型艺术表演很有韵味...      0
2  槟榔谷，大家都知道槟榔，也知道海南人有吃槟榔的习惯，初到三亚，就听说出租车司机朝窗外吐槟榔，...      0
3  黎苗族文化主题公园，展示黎族、苗族服饰手工文化，传统民居，狩猎技能，传统舞蹈，民族美食，环境...      0
4  去的那天太阳很大，带着儿子一起去的，里面主要是少数民族的传统习俗文化遗产等，导游讲解一下会更...      0

In [3]:
df = pd.read_csv('csv/test.csv')
df.head()

content  score
0  "整个行程安排非常合理，性价比很高！对湘西有了新的认识，张家界和凤凰真的很值得！导游小哥服务...      0
1  "本来打算自助游，但看了网上热评后果断选择了这个团，结果证实是明智之举。周导是地地道道的土家...      0
2  "行程安排完全符合购买的内容，说实话还是有点累的，但也能理解毕竟天数少去的地方多。景色方面，...      0
3  "行程安排很合理，会全部安排到位，吃得好，住得好，玩得爽，真心不错，没来玩过的朋友***，玩...      0
4                               '这次出游感觉导游水平很高，行程安排合理      1

In [4]:
a = df['content']

In [5]:
df['content']=load_corpus(a)
print(df)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Aris\AppData\Local\Temp\jieba.cache
Loading model cost 0.821 seconds.
Prefix dict has been built successfully.


                                                content  score
0     槟榔 谷 的 大门 一左一右 两个 大 牛头 霸气 侧漏 的 感觉 溢于言表 拿 着 预订 ...      1
1     海南 保亭 的 黎 苗族 风情 景区 可以 看到 他们 的 艺术 居住 环境 房屋 传统 表...      0
2     槟榔 谷 大家 都 知道 槟榔 也 知道 海南 人有 吃 槟榔 的 习惯 初到 三亚 就 听...      0
3     黎 苗族 文化 主题公园 展示 黎族 苗族 服饰 手工 文化 传统 民居 狩猎 技能 传统 ...      0
4     去 的 那天 太阳 很大 带 着 儿子 一起 去 的 里面 主要 是 少数民族 的 传统习俗...      0
...                                                 ...    ...
6244                               夏天 海南 太热 了 去 海边 开心 吧      0
6245                                  海滩 还 不错 帅哥美女 也 不错      0
6246                                很 适合 游泳 的 地方 大海 很 美      0
6247                                    海滩 不错 的 免费 对外开放      0
6248                                  就 在 酒店 旁边 很美 很 方便      0

[6249 rows x 2 columns]


## 随机取样划分训练集与测试集

In [17]:
df_train = df.sample(frac=0.9, replace=False, random_state=0, axis=0)
print(df_train)
df_test = df[~df.index.isin(df_train.index)]
print(df_test)

                                                content  score
2600  不错 的 景点 之一 不错 的 选择 哦 不错 的 景点 之一 感觉 很 好 的 选择 哦 ...      0
5997                  海是 琼州海峡 可以 看到 徐闻 但水 不算 干净 其他 还好 吧      0
3147                             景区 干净 大气 玉带 滩 让 人 不可思议      0
144   特别 值得 一去 的 槟榔 谷 浓郁 的 少数民族 民风 还有 超级 舒服 的 负氧离子 简...      0
1525            单买 门票 性价比 不错 演出 实在 一般 不值 这个 钱 而且 后排 看不见      1
...                                                 ...    ...
830   三亚 最 值得 去 的 地方 原声态 景区 带 着 父母 去 旅游 可惜 那天 开会 会场 ...      1
5046          打破 往日 的 宁静 这里 正在 举办 沙滩 马拉松 比赛 人来人往 很 是 热闹      0
5891                            漂亮 的 海滩 儿子 耍 疯 了 不想 走 了      0
3418                春节 全家 一起 去 旅行 在 携程 上 买 的 票 取票 方便 不错      1
1734  非常 值得 一去 到 三亚 看海 是 众所周知 的 但是 黎 文化 一颗 璀璨 的 明珠 整...      0

[5624 rows x 2 columns]
                                                content  score
0     槟榔 谷 的 大门 一左一右 两个 大 牛头 霸气 侧漏 的 感觉 溢于言表 拿 着 预订 ...      1
21    很 难忘 的 一次 旅途 让 我 见识 了 黎苗 的 文化 黎家 的 阿哥 阿妹 热情好客 ...      0
24    说实话 这个 景区 我来 过 两次 两次 下来 

## 特征编码

In [18]:
stopwords = []
with open("stop.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
x_train = vectorizer.fit_transform(df_train['content'])
y_train = df_train['score']

C:\Users\Aris\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a', 'ain', 'aren', 'c', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'i', 'isn', 'lex', 'll', 'm', 'mon', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn', 'β', 'δ', 'λ', 'ξ', 'ψ', 'в', 'ⅲ', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '元', '吨', '数', '日', '末', '０', '１', '１２', '２', '３', '５', 'ａ', 'ｂ', 'ｃ', 'ｅ', 'ｆ', 'ｌ', 'ｌｉ', 'ｒ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


In [20]:
x_test = vectorizer.transform(df_test['content'])
y_test = df_test['score']

## 模型训练

In [21]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [22]:
y_pre = clf.predict(x_test)
print(y_pre)

[1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [23]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pre))
print('准确率:', metrics.accuracy_score(y_test, y_pre))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91       481
           1       0.81      0.45      0.58       144

    accuracy                           0.85       625
   macro avg       0.83      0.71      0.74       625
weighted avg       0.85      0.85      0.83       625

准确率: 0.8496


## 测试

In [24]:
pddd=pd.read_csv('csv/test.csv')
xx_test = vectorizer.transform(pddd['content'])
yy_test = pddd['score']
yy_pre = clf.predict(xx_test)
from sklearn import metrics

print(metrics.classification_report(yy_test, yy_pre))
print('准确率:', metrics.accuracy_score(yy_test, yy_pre))

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       156
           1       1.00      0.05      0.09       152

    accuracy                           0.53       308
   macro avg       0.76      0.52      0.39       308
weighted avg       0.76      0.53      0.39       308

准确率: 0.5292207792207793


In [32]:
str = [
    "这里值得一游：1）景色很美，节目表演完整讲述黎族人生活习惯和服饰；2）景区导游路线设计合理，介绍全面；景区内即有实物，又有博物馆；生动介绍黎、苗两族和谐相处的生活；3）景区导游服务态度和蔼可亲，特别给我们做解说的程杰导游，知识渊博，解说详细，让我们很好的了解黎、苗两族的发展历史。80元钱花的不冤枉。不愧为中国5A级景区称号。，总体感觉还行，期待不高所以失望也不大，会址外观和会议厅高端大气，高尔夫球场绿地看着清新怡人，玉带滩确是奇幻，一边河水平静，一边海浪滚滚拍岸，只是商业味太重，很多小贩推销东西，有些乱糟糟"]
words = [processing(s) for s in str]
vec = vectorizer.transform(words)
output = clf.predict(vec)
#  0 好评 1 差评
print(output)

[0]


In [29]:
pdd = pd.read_csv('csv/test.csv')
pdd_test  = pdd['content']
pdd_label = pdd['score']
train_result=[]
train_score = 0 # 测评准确率
pdd_index = 0
for i in pdd_test:
    str = i
    words = [processing(s) for s in str]
    vec = vectorizer.transform(words)
    output = clf.predict(vec)
    pdd.loc[pdd_index,'t_score'] = output[0]
    pdd_index +=1
print(pdd)

                                               content  score  t_score
0    "整个行程安排非常合理，性价比很高！对湘西有了新的认识，张家界和凤凰真的很值得！导游小哥服务...      0      0.0
1    "本来打算自助游，但看了网上热评后果断选择了这个团，结果证实是明智之举。周导是地地道道的土家...      0      0.0
2    "行程安排完全符合购买的内容，说实话还是有点累的，但也能理解毕竟天数少去的地方多。景色方面，...      0      0.0
3    "行程安排很合理，会全部安排到位，吃得好，住得好，玩得爽，真心不错，没来玩过的朋友***，玩...      0      0.0
4                                 '这次出游感觉导游水平很高，行程安排合理      1      0.0
..                                                 ...    ...      ...
303  时间安排成问题，各种等待，各种坑。早上五点起来说滑雪场排队。可等导游等了2个多小时。导游素质...      1      0.0
304  在宜昌市区到座船的路上导游说两天的中餐和第一天晚餐不包含在团费里面，可以自己在外面吃，但是得...      1      0.0
305  神农架景色不错，但旅行社服务不怎么样，第一天武汉的导游订的九点多到宜昌的票，宜昌的旅行社要求...      1      0.0
306  预定的两天行程，结果因为什么兴山县修路不让旅游大巴车走，硬是把我们在游船上困了3个小时左右，...      1      1.0
307  風景不錯,行程安排也行,就是酒店不太乾淨,回程火車票安排太差,七點就到宜昌站,給的是9點多的...      1      1.0

[308 rows x 3 columns]


In [30]:
print(pdd['score'][0] == pdd['t_score'][0])

True


In [31]:
train_score=0
for ii in range(0,308):
    if pdd['score'][ii] == pdd['t_score'][ii]:
        # print(pdd['score'][ii],pdd['t_score'][ii])
        train_score +=1
    else:
        print(pdd.loc[ii])
print('准确率:',train_score,train_score/308)

content    '这次出游感觉导游水平很高，行程安排合理
score                         1
t_score                     0.0
Name: 4, dtype: object
content    不建议国内小长假来龙门和少林寺体验实在不佳唯独这季节盛开的牡丹花倾国倾城也算安抚了一下我的心情
score                                                    1
t_score                                                0.0
Name: 157, dtype: object
content    景区问题:龙门石窟管理混乱，检票口，电瓶车都没要团散分开，通行速度特别慢，耽误时间。小宋城吃...
score                                                      1
t_score                                                  0.0
Name: 158, dtype: object
content    首先国庆出游路上非常堵，景点人非常多，这个我们第一次去的游客不了解情况，但是携程与当地合作的...
score                                                      1
t_score                                                  0.0
Name: 159, dtype: object
content    整个出行还算顺利，导游也是忙前忙后服务比较周到。就是我一开始上的大巴，到了目的地就分去别的导...
score                                                      1
t_score                                                  0.0
Name: 160, dtype: object
content    导游还不错，酒店也不错，但是不是真正的纯玩团。最后一天上午大概逛凤凰半小时，购物点却逛了